## Req 1: Khuyến nghị sản phẩm với Collaborative Filtering

## Setup pyspark

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!cp /content/drive/MyDrive/spark-3.1.1-bin-hadoop3.2.tgz .
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [5]:
import findspark
findspark.init()

In [6]:
import pyspark as spark

print(spark.__version__)

3.1.1


### Library

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
import matplotlib.pyplot as plt
import csv
import os
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

### Process

In [ ]:
df = pd.read_csv('/content/ratings2k.csv')
#Spit data X_text = 0.3, X_Train = 0.7
X_train, X_test = train_test_split(df, test_size=0.3, random_state=42)

In [28]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Collaborative Filtering Recommendation System") \
    .getOrCreate()

# Load the dataset
data = spark.read.csv("ratings2k.csv", header=True, inferSchema=True)

# Display the schema of the dataset
data.printSchema()

# Divide the dataset into training and test sets (70:30 ratio)
(training, test) = data.randomSplit([0.7, 0.3], seed=123)
# Initialize lists to store MSE values and number of similar users
mse_values = []
similar_users_values = list(range(10, 21))

root
 |-- index: integer (nullable = true)
 |-- user: integer (nullable = true)
 |-- item: integer (nullable = true)
 |-- rating: double (nullable = true)



In [42]:
test.drop('rating')

DataFrame[index: int, user: int, item: int]

In [36]:
als = ALS(rank=10, seed=0)
als.setMaxIter(5)
als.getMaxIter()
als.setRegParam(0.1)
als.getRegParam()
als.clear(als.regParam)
model = als.fit(training)
model.getBlockSize()

4096

In [39]:
model.setPredictionCol("newPrediction")
model.userFactors.orderBy("id").collect()

10

In [43]:
predictions = sorted(model.transform(test).collect(), key=lambda r: r[0])

In [47]:
predictions

[Row(index=2, user=72, item=26, rating=1.0, newPrediction=4.082702159881592),
 Row(index=6, user=72, item=426, rating=4.0, newPrediction=3.951749563217163),
 Row(index=9, user=59, item=22, rating=3.5, newPrediction=2.7017791271209717),
 Row(index=12, user=26, item=322, rating=3.5, newPrediction=3.775352954864502),
 Row(index=13, user=8, item=416, rating=4.0, newPrediction=nan),
 Row(index=18, user=9, item=455, rating=4.0, newPrediction=3.0461559295654297),
 Row(index=23, user=13, item=128, rating=5.0, newPrediction=3.7721431255340576),
 Row(index=29, user=59, item=176, rating=4.0, newPrediction=3.367406129837036),
 Row(index=30, user=72, item=184, rating=4.0, newPrediction=3.559579610824585),
 Row(index=33, user=64, item=144, rating=5.0, newPrediction=4.273157119750977),
 Row(index=34, user=25, item=81, rating=4.0, newPrediction=3.2477643489837646),
 Row(index=35, user=25, item=277, rating=3.0, newPrediction=3.1594653129577637),
 Row(index=38, user=22, item=216, rating=4.0, newPredicti

# Req 2: Dự đoán giá chứng khoán.